In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

In [ ]:
import os
import torch
import pandas as pd
os.environ['HF_HOME'] = '/home/tl688/scratch/'
os.environ['HF_TOKEN'] = 'your token'

In [ ]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "./reason_model/Path_Reason/", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("./reason_model/Path_Reason/")

In [ ]:
df_isin_select = pd.read_csv("./qa_pathinstruct_withreason.csv")

In [ ]:
df_isin_select['reason'].values[0]

In [ ]:
QUESTION_TEMPLATE = """{question} 
Your task: 
1. Think through the question step by step, enclose your reasoning process in <think>...</think> tags. 
2. Then provide the correct single-letter choice (A, B, C, D,...) inside <answer>...</answer> tags.
3. No extra information or text outside of these tags."""

In [ ]:
import pickle
from glob import glob
data_test = []

for item in glob("./output_reason/*"):
    data_test.append(item)

In [ ]:
image_path = data_test[0]
print(image_path)

In [ ]:
df_reneeq = pd.read_json("./PathGen-Instruct.json")

In [ ]:
df_reneeq['image'].values[0]

In [ ]:
df_rest = df_reneeq[df_reneeq['image'] == 'TCGA-CH-5750-01Z-00-DX1.88db9d82-82c0-44b7-87b2-6e3bcdfb58ad/120384_6112.jpg']

In [ ]:
question = df_rest['conversations'].values[0][0]['value']

In [ ]:
question

In [ ]:
df_rest['conversations'].values

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": QUESTION_TEMPLATE.format(question=question)},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=256)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)